In [12]:
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

import numpy as np

import pandas as pd


In [13]:
# carregar modelo word2vec
word2vec = KeyedVectors.load_word2vec_format('../skip_s100.txt')
vocab_size, embedding_size = word2vec.vectors.shape

In [14]:
# carregar dados de treinamento do CSV
df = pd.read_csv('../train.csv', header=None, names=['texto', 'classe'])
X = df['texto'].values
y = df['classe'].values

In [15]:
# tokenizar o texto
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# padronizar o tamanho das sequencias
max_length = max([len(text) for text in X])
X = pad_sequences(X, maxlen=max_length, padding='post')

In [16]:
# construir modelo
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, weights=[word2vec.vectors], input_length=max_length, trainable=False))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Dense(7, activation='softmax'))

In [17]:
# compilar modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
# separar dados em treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, 7)
y_val = to_categorical(y_val, 7)

In [19]:
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=7,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

In [20]:
# treinar modelo
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping_monitor])

Epoch 1/30
26/26 [==============================] - 5s 41ms/step - loss: 1.5433 - accuracy: 0.5530 - val_loss: 0.7936 - val_accuracy: 0.7291
Epoch 2/30
26/26 [==============================] - 0s 12ms/step - loss: 0.5157 - accuracy: 0.8116 - val_loss: 0.3575 - val_accuracy: 0.8966
Epoch 3/30
26/26 [==============================] - 0s 12ms/step - loss: 0.2707 - accuracy: 0.9138 - val_loss: 0.3263 - val_accuracy: 0.8768
Epoch 4/30
26/26 [==============================] - 0s 12ms/step - loss: 0.1705 - accuracy: 0.9470 - val_loss: 0.2411 - val_accuracy: 0.9310
Epoch 5/30
26/26 [==============================] - 0s 12ms/step - loss: 0.1199 - accuracy: 0.9667 - val_loss: 0.2447 - val_accuracy: 0.9360
Epoch 6/30
26/26 [==============================] - 0s 12ms/step - loss: 0.1323 - accuracy: 0.9667 - val_loss: 0.2801 - val_accuracy: 0.9409
Epoch 7/30
26/26 [==============================] - 0s 12ms/step - loss: 0.0696 - accuracy: 0.9840 - val_loss: 0.2251 - val_accuracy: 0.9360
Epoch 8/30
26